### Links
> IBGE censo https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html
------------------------
> Prefeitura SP projeções https://www.prefeitura.sp.gov.br/cidade/secretarias/licenciamento/desenvolvimento_urbano/dados_estatisticos/info_cidade/demografia/index.php?p=260265


In [1]:
import $ivy.`org.apache.spark::spark-core:3.1.2`
import $ivy.`org.apache.spark::spark-sql:3.1.2`
import $ivy.`org.apache.hadoop:hadoop-client:3.1.2`
import $ivy.`org.apache.hadoop:hadoop-aws:3.1.2`
import $ivy.`com.amazonaws:aws-java-sdk-s3:1.11.375`
import $ivy.`com.amazonaws:aws-java-sdk-core:1.11.375`


import $ivy.$                                   

import $ivy.$                                  

import $ivy.$                                      

import $ivy.$                                   

import $ivy.$                                       

import $ivy.$                                         


In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
    import org.apache.log4j.{Level, Logger}
    Logger.getLogger("org").setLevel(Level.OFF)
      SparkSession.builder()
        .master("local[3]")
        .config("spark.sql.warehouse.dir", "s3://fernando-bontorin-censo")
        .getOrCreate()
}

import spark.implicits._
import org.apache.spark.sql.expressions.Window

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@53b3d1d2
import spark.implicits._

import org.apache.spark.sql.expressions.Window

In [3]:
val estabs = spark.read.options(
    Map("inferSchema" -> "true", "header" -> "true", "delimiter" -> ",", "encoding" -> "UTF-8")
).csv("s3a://fernando-bontorin-datasus/tbEstabelecimentoFull")

estabs.printSchema

root
 |-- CO_UNIDADE: long (nullable = true)
 |-- CO_CNES: integer (nullable = true)
 |-- NU_CNPJ_MANTENEDORA: long (nullable = true)
 |-- TP_PFPJ: integer (nullable = true)
 |-- NIVEL_DEP: integer (nullable = true)
 |-- NO_RAZAO_SOCIAL: string (nullable = true)
 |-- NO_FANTASIA: string (nullable = true)
 |-- NO_LOGRADOURO: string (nullable = true)
 |-- NU_ENDERECO: string (nullable = true)
 |-- NO_COMPLEMENTO: string (nullable = true)
 |-- NO_BAIRRO: string (nullable = true)
 |-- CO_CEP: integer (nullable = true)
 |-- NU_CPF: string (nullable = true)
 |-- NU_CNPJ: long (nullable = true)
 |-- CO_ATIVIDADE: integer (nullable = true)
 |-- CO_CLIENTELA: integer (nullable = true)
 |-- TP_UNIDADE: integer (nullable = true)
 |-- CO_ESTADO_GESTOR: integer (nullable = true)
 |-- CO_MUNICIPIO_GESTOR: integer (nullable = true)
 |-- TP_ESTAB_SEMPRE_ABERTO: string (nullable = true)
 |-- CO_TIPO_UNIDADE: string (nullable = true)
 |-- NO_FANTASIA_ABREV: string (nullable = true)
 |-- TP_GESTAO: strin

estabs: DataFrame = [CO_UNIDADE: bigint, CO_CNES: int ... 24 more fields]

In [7]:
estabs.select(col("CO_UNIDADE"), col("competencia"), substring(col("competencia"), 0, 4).as("ano"), substring(col("competencia"), 6, 2).as("mes")).show(2)

+-------------+-----------+----+---+
|   CO_UNIDADE|competencia| ano|mes|
+-------------+-----------+----+---+
|3550306578632| 2017-08-01|2017| 08|
|3550306580955| 2017-08-01|2017| 08|
+-------------+-----------+----+---+
only showing top 2 rows



In [12]:
estabs
.withColumn("ano", substring(col("competencia"), 0, 4))
.withColumn("mes", substring(col("competencia"), 6, 2))
.where($"mes" === "12")
.coalesce(1)
.repartitionByRange(col("ano"))
.write
.mode(SaveMode.Overwrite)
.options(
    Map("header" -> "true", "delimiter" -> ",", "encoding" -> "UTF-8")
)
.csv("s3a://fernando-bontorin-datasus/tbEstabelecimentoAno")